# Cryptography - OpenSSL mini-project

Nima Dekhli 
25 January 2024

## Constraints

- asymmetric algorithm : RSA
- symmetric algorithm : AES, mode OFB 
- signature : DSA 
- hash : SHA-512 

In [7]:
import os

from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding, utils
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [8]:
class Person:
    RSA_KEY_SIZE = 2048
    RSA_KEY_EXPONENT = 65537
    HASH_ALGORITHM = hashes.SHA512()
    AES_KEY_SIZE_BITS = 256

    def __init__(self, name):
        """
        Creates a new person and generates their RSA key pair
        :param name: the name of the person
        """
        self.name = name
        self.key = self._generate_rsa_key_pair()
        self.public_keys_directory: dict[str, rsa.RSAPublicKey] = {}

        self.aes_key = None

    @classmethod
    def _generate_rsa_key_pair(cls):
        """
        Generates a new RSA key pair
        :return rsa.RSAPublicKey: The generated private key
        """
        return rsa.generate_private_key(cls.RSA_KEY_EXPONENT, cls.RSA_KEY_SIZE)

    def import_public_key(self, name, public_key, force=False):
        """
        Imports a public key in the directory
        :param str name: The name of the owner of the public key
        :param rsa.RSAPublicKey public_key: The public key to import
        :param bool force: True to overwrite an existing key
        :return None:
        """
        if name in self.public_keys_directory and not force:
            raise Exception("Error: a public key with this name "
                            "is already imported. Use force=True to overwrite it")

        if name in self.public_keys_directory and force:
            print("Warning: overwriting existing public key")

        self.public_keys_directory[name] = public_key

    def get_public_key(self):
        """
        Returns the public key of the person
        :return rsa.RSAPublicKey: The public key
        """
        return self.key.public_key()

    def generate_new_aes_key(self):
        """
        Generates a new random AES key
        :return bytes: The generated AES key
        """
        self.aes_key = os.urandom(self.AES_KEY_SIZE_BITS // 8)

    @classmethod
    def get_random_iv(cls):
        """
        Generates a new random initialization vector
        :return bytes: The generated IV
        """
        return os.urandom(16)

    def import_aes_key(self, key, force=False):
        """
        Imports an AES key
        :param force: True to overwrite an existing key
        :param bytes key: The AES key to import
        :return None:
        """
        if self.aes_key is not None and force:
            print("Warning: overwriting existing AES key")

        elif self.aes_key is not None and not force:
            raise Exception("Error: an AES key is already imported. Use force=True to overwrite it")

        self.aes_key = key

    def aes_encrypt(self, message):
        """
        Encrypts the given data with the existing AES key
        :param bytes|str message: The data to encrypt
        :return tuple[bytes, bytes]: The encrypted data and the initialization vector
        """
        assert self.aes_key is not None, "You must generate or import an AES key first"

        if type(message) is str:
            message = message.encode('utf-8')

        iv_ = self.get_random_iv()
        cipher = Cipher(algorithms.AES256(self.aes_key), modes.OFB(iv_))
        encryptor = cipher.encryptor()

        return encryptor.update(message) + encryptor.finalize(), iv_

    def aes_decrypt(self, message, iv):
        """
        Decrypts the given data with the existing AES key and given initialization vector
        :param bytes message: The data to decrypt
        :param bytes iv: The initialization vector to use
        :return bytes: The decrypted data
        """
        assert self.aes_key is not None, "You must generate an AES key first"

        cipher = Cipher(algorithms.AES256(self.aes_key), modes.OFB(iv))
        decryptor = cipher.decryptor()

        return decryptor.update(message) + decryptor.finalize()

    def get_aes_key(self):
        """
        Returns the AES key
        :return bytes: The AES key
        """
        assert self.aes_key is not None, "You must generate an AES key first"
        return self.aes_key

    def sign(self, message=None, hash=None):
        """
        Signs the given data
        :param bytes message: The data to sign
        :return bytes: The signature
        """

        assert message is not None or hash is not None, \
            "You must specify either the message or the hash of the message"

        if hash is None:
            hash = hashes.Hash(self.HASH_ALGORITHM)
            hash.update(message)
            hash = hash.finalize()

        return self.key.sign(
            hash,
            padding.PSS(
                mgf=padding.MGF1(self.HASH_ALGORITHM),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            utils.Prehashed(self.HASH_ALGORITHM)
        )

    def verify(self, signature, message=None, hash=None, name=None, public_key=None):
        """
        Verify the given signature for the given message.
        You can either give the message or the hash of the message, but not both.
        You can either give the name of the sender or their public key, but not both.
        If you give their name, the public key must have been imported before.

        :param bytes signature: The signature to verify
        :param bytes message: The message to verify
        :param bytes hash: The hash of the message to verify
        :param str name: The name of the sender
        :param public_key: The public key of the sender
        :return: True if the signature is valid, False if not
        """
        assert message is not None or hash is not None, \
            "You must specify either the message or the hash of the message"

        assert message is None or hash is None, \
            "You must specify either the message or the hash of the message, not both"

        assert public_key is not None or name is not None, \
            "You must specify either the public key or the name of the sender"

        assert public_key is None or name is None, \
            "You must specify either the public key or the name of the sender, not both"

        # if the hash is not given, we compute it
        if hash is None:
            hash = hashes.Hash(self.HASH_ALGORITHM)
            hash.update(message)
            hash = hash.finalize()

        # if the public key is not given, we get it from the directory
        if name is not None:
            assert name in self.public_keys_directory, \
                "The name of the sender is not in the directory"
            public_key = self.public_keys_directory[name]

        # we verify the signature
        try:
            public_key.verify(
                signature,
                hash,
                padding.PSS(
                    mgf=padding.MGF1(self.HASH_ALGORITHM),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                utils.Prehashed(self.HASH_ALGORITHM)
            )
            # signature verification succeeded because no exception was raised
            return True
        except:
            # signature verification failed
            return False
        
    
    def verify_and_print(self, *args, **kwargs):
        if self.verify(*args, **kwargs):
            print("[ OK ] Signature is valid")
        else:
            print("[ ERROR ] *** SIGNATURE VERIFICATION FAILED! ***")


    def asymmetric_encrypt(self, message, public_key=None, name=None):
        """
        Encrypts the given data with the given public key or the public key of the given name.
        Either the public key or the name must be given, but not both.

        :param string|bytes message: The message to encrypt
        :param public_key: The public key to use
        :param name: The name of the recipient. Must have been imported before.
        :return: The encrypted data
        """
        assert public_key is not None or name is not None, \
            "You must specify either the public key or the name of the recipient"

        if name is not None:
            assert name in self.public_keys_directory, \
                "The name of the recipient is not in the directory"
            public_key = self.public_keys_directory[name]

        return public_key.encrypt(
            message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA512()),
                algorithm=hashes.SHA512(),
                label=None)
        )

    def asymmetric_decrypt(self, message):
        """
        Decrypts the given data with the person's private key
        :param message: The data to decrypt
        :return: The decrypted data
        """
        return self.key.decrypt(
            message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=self.HASH_ALGORITHM),
                algorithm=self.HASH_ALGORITHM,
                label=None)
        )

    def __str__(self):
        """
        Returns a string representation of the person with their name and public key
        :return: The string representation
        """
        return (self.name + ":\n" +
                self.key.public_key().public_bytes(
                    serialization.Encoding.PEM,
                    serialization.PublicFormat.SubjectPublicKeyInfo).decode('utf-8')
                )


## Exemple

### RSA key generation and exchange

First, we create Alice and Bob 

In [ ]:
alice = Person("alice")
bob = Person("bob")

Their RSA keys are generated automatically.

In [9]:
print(alice)
print(bob)

alice:
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAscD5bBbiA58/eyVbXpN0
KrIqs9T4yAPJHl7L9amjT9IFnoB8dWp0L+Acj3P0+Cu2lTnFs+i4iesNvE4jB4Yv
aUDkdhq8Ia4KhQBFvaw/s5UFBGG84zsiC8qCU28C+pkDDte4R9aIessmxyQ9NBW2
TEfGjAuIVtAebJ98MZ3S2Pdl0HtwGNIAdJVuJ8aRGkqD9EtzR0RpQ5jUIN0YM/fq
EV+yOcIL4lzyh8ot9+MSMoUkvNk4K2b3yNci3yWKlG9OY+wcWhihwY53cC4pc1yA
sEFHKBCQ5kmOTSCe6Jm2cyruIykKWk1lDWuFxBQrkyoeyOb0OoPVOc66QVEOiymp
FwIDAQAB
-----END PUBLIC KEY-----

bob:
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA7Y6pmPRrv64MZzuqlMOF
ltadUw7eWBwqiPb5v+N4n4f/ilCT3oLqtjg5dr4SnFs4XFfSnALVs6+/VqeOvkwV
GJCtSDt5uTgsRzGGxQDihe5t8vq+JI2gkZkasP27W6zEDo5Q4X3r+PWOf8h599F/
vPgvW+mZij1kD/z5quLKb+r/ME5Ft04yOvX53yhI4inF57YqOlS+YAFmZoXR5sLt
0zZMtCLXlpbAcgvsqlGXeh7lAGwv4pFu1zMVv/rXVhlmF67rj6DGAKKxAMrWawoc
eaUuYf5HHeiLnv0nxRQ+CkNzgKtCLjJRRxb22fhyxXbpnuMN6AGVSuwrU4DQ5oD7
cQIDAQAB
-----END PUBLIC KEY-----



So now, we can import their public keys in each other's directory.

In [10]:
alice_public_key = alice.get_public_key()
bob_public_key = bob.get_public_key()

alice.import_public_key("bob", bob_public_key)
bob.import_public_key("alice", alice_public_key)

Alice now generates a new AES key that will be stored in her object.

In [11]:
alice.generate_new_aes_key()

She wants to send it to Bob, but she wants to make sure that Bob is the only one who can read it.
Moreover, Bob must be sure that the message comes from Alice.

She now has two options : 
- either she first signs the message and then encrypts it
- or she first encrypts the message and then signs it

Both options are valid . 

### Option 1 : sign then encrypt

Alice first sign the message. To do so, she uses her private key.

In [12]:
aes_plain_key = alice.get_aes_key()
aes_plain_key_signature = alice.sign(aes_plain_key)

Then, she encrypts the message with Bob's public key. Notice that we only 
need to provide the name of the recipient as we have already imported his public key.

In [13]:
aes_encrypted_key = alice.asymmetric_encrypt(aes_plain_key, name="bob")

print(f"Plain AES key : {aes_plain_key.hex()}")
print(f"Encrypted AES key : {aes_encrypted_key.hex()}")

Plain AES key : 34c7197890685cf656b08fd72a818b0c547dc5082f920fcd996326716dd0a6c9
Encrypted AES key : be64b7c3315e21a61e9a24c5b422c34c666426ae9d1743aa63d3da793758df6515337c485daa8c43c4ddee52042373cb8d46cb1f8ba9a5f825f3a7757dc83393ec9f6bda620b3413aff714d055d3b02afd26b510be411b36c45654037c3a2eef9fec244441ac9006e8c2cf942cfa5ace7488b2f3d5bdfa7cdc1104f74350b636a5a67d4d2c33411138ff654e349cc8e5d2d20cf17c1819d69df7dcc262765367c95b05700978b8528f7405888a317ccf1f842ec6dce7b4abd6ed11c6ed49966d6cc6a4d445c878bf63f591f55b114df70f603f866102ed0b8f6288f04c05d305cf87a22320c2c48dc4109f1308816c4f8783e136126d1788c91375d3c2be7523


Now, Alice can send the encrypted AES key and the signature to Bob.

As Alice first signed the message and only then encrypted it, Bob must first decrypt the message and then verify the signature. 

In [14]:
aes_decrypted_key = bob.asymmetric_decrypt(aes_encrypted_key)

print(aes_decrypted_key.hex())

34c7197890685cf656b08fd72a818b0c547dc5082f920fcd996326716dd0a6c9


As we see, the decrypted AES key is the same as the one Alice generated.

Now Bob can verify the signature.

In [15]:

bob.verify_and_print(signature=aes_plain_key_signature, message=aes_decrypted_key, name='alice')

[ OK ] Signature is valid


Let's assume that Eve intercepted the message. She cannot alter it because 
it is encrypted. Decryption will fail : 

In [16]:
eve = Person("eve")
eve.import_public_key("alice", alice_public_key)
eve.import_public_key("bob", bob_public_key)

In [17]:
aes_encrypted_key_altered = aes_encrypted_key[:-1] + b'\x00'

In [18]:
aes_decrypted_key_altered = bob.asymmetric_decrypt(aes_encrypted_key_altered)

ValueError: Encryption/decryption failed.

But what she could do is to replace the encrypted AES key with another one.

In [19]:
eve.generate_new_aes_key()
aes_plain_key_signature_eve = eve.sign(eve.get_aes_key())
aes_encrypted_key_eve = eve.asymmetric_encrypt(eve.get_aes_key(), name="bob")

Now Bob received the altered data given by Eve, even though he doesn't know it and
thinks that it comes from Alice.

First, he decrypts the message.

In [20]:
aes_decrypted_key_eve = bob.asymmetric_decrypt(aes_encrypted_key_eve)

print(aes_decrypted_key_eve.hex())

2aeb690587f68e7d415d437a8d3a4de08d58a13be8c736637aae5e2a779ef799


We can see that the decrypted AES key is not the same as the one Alice generated 
but Bob has no way to know that. 

But now, Bob can verify the signature **with Alice's public key**.

In [21]:
bob.verify_and_print(signature=aes_plain_key_signature_eve, message=aes_decrypted_key_eve, name='alice')

[ ERROR ] *** SIGNATURE VERIFICATION FAILED! ***


As we can see, the signature is invalid! Bob knows that the message has been altered.

### Option 2 : encrypt then sign

In [22]:
aes_plain_key = alice.get_aes_key()
aes_encrypted_key = alice.asymmetric_encrypt(aes_plain_key, name="bob")
aes_encrypted_key_signature = alice.sign(aes_encrypted_key)

In [23]:
bob.verify_and_print(signature=aes_encrypted_key_signature, message=aes_encrypted_key, name='alice')

[ OK ] Signature is valid


In [24]:
aes_decrypted_key = bob.asymmetric_decrypt(aes_encrypted_key)
print(aes_decrypted_key.hex())

34c7197890685cf656b08fd72a818b0c547dc5082f920fcd996326716dd0a6c9


Bob can now safely import the AES key in his object. From now on 
he can use it to encrypt and decrypt messages with Alice (and no one else).

In [25]:
bob.import_aes_key(aes_decrypted_key)

Now, let's assume that Eve intercepted the message. She could alter it by adding 
one byte at the end of the encrypted AES key.

In [26]:
aes_encrypted_key_altered = aes_encrypted_key + b'\x00'

Before decrypting the message, Bob must verify the signature.

In [27]:
bob.verify_and_print(signature=aes_encrypted_key_signature, message=aes_encrypted_key_altered, name='alice')

[ ERROR ] *** SIGNATURE VERIFICATION FAILED! ***


As we can see, the signature is invalid. Bob knows that the message has been altered and 
does not have to decrypt it.

## Using the AES key to exchange messages

Let's assume Bob wants to send a message to Alice. He first encrypts it with the AES key
he just received and imported. 

In [28]:
message = "Hello Alice! This is Bob speaking. How are you?"
encrypted_message, iv = bob.aes_encrypt(message)

print(f"Encrypted message : {encrypted_message.hex()}")
print(f"Initialization vector : {iv.hex()}")

Encrypted message : 05c8d9f4156092fa8621babdd98062db0b16f13e8bf7bad6e7a2caf8c7fd3db24fd0ba98731b92e15305e07798dfb6
Initialization vector : 077e1a71e6d0b8a8f7de69b6a5dd5f9e


For each message, we generate a random initialization vector 
that will be used to encrypt and decrypt the message. It 
does not need to be secret and can be sent in clear.

Alice can now decrypt the message

In [29]:
decrypted_message = alice.aes_decrypt(encrypted_message, iv)
print(decrypted_message.decode('utf-8'))

Hello Alice! This is Bob speaking. How are you?


As we can see, the message has been decrypted correctly. Alice 
can now answer to Bob with the same AES key.
However, the initialization vector must be different for each message.

In [30]:
message = "Hello Bob! I'm fine, thanks. What about you?"
encrypted_message, iv = alice.aes_encrypt(message)
print(f"Encrypted message : {encrypted_message.hex()}")
print(f"Initialization vector : {iv.hex()}")


Encrypted message : 9cb93512f36653cde218f5c27cae2ab93841b055f450c26b68c210f712b66e8d23f4feea7ca3de6c2d0c8818
Initialization vector : e0ce72d9cfb98f670ae05cfa84ebae03


In [ ]:
decrypted_message = bob.aes_decrypt(encrypted_message, iv)
print(decrypted_message.decode('utf-8'))

## ChatGPT solution

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def generate_rsa_keypair():
    """ Génère une paire de clés RSA """
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def serialize_public_key(public_key):
    """ Sérialise la clé publique pour un transport sûr """
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    return pem

def encrypt_rsa(public_key, data):
    """ Chiffre des données avec la clé publique RSA """
    return public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA512()),
            algorithm=hashes.SHA512(),
            label=None
        )
    )

def decrypt_rsa(private_key, data):
    """ Déchiffre des données avec la clé privée RSA """
    return private_key.decrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA512()),
            algorithm=hashes.SHA512(),
            label=None
        )
    )

def sign_data(private_key, data):
    """ Signe des données avec une clé privée RSA """
    return private_key.sign(
        data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA512()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA512()
    )

def verify_signature(public_key, signature, data):
    """ Vérifie la signature avec une clé publique RSA """
    try:
        public_key.verify(
            signature,
            data,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA512()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA512()
        )
        return True
    except Exception:
        return False

def encrypt_aes(key, data):
    """ Chiffre des données avec AES en mode OFB """
    cipher = AES.new(key, AES.MODE_OFB)
    iv = cipher.iv
    ciphertext = cipher.encrypt(data)
    return iv, ciphertext

def decrypt_aes(key, iv, data):
    """ Déchiffre des données avec AES en mode OFB """
    cipher = AES.new(key, AES.MODE_OFB, iv=iv)
    return cipher.decrypt(data)


# Génération des clés RSA pour Alice et Bob
alice_keypair = generate_rsa_keypair()
bob_keypair = generate_rsa_keypair()
print("Clés RSA générées pour Alice et Bob.")

# Alice génère une clé AES pour chiffrer les messages
aes_key = get_random_bytes(16)  # Clé AES 128 bits
print("Clé AES générée par Alice.")

# Alice envoie la clé AES à Bob, chiffrée avec la clé publique RSA de Bob
encrypted_aes_key = encrypt_rsa(bob_keypair.public_key(), aes_key)
print("Alice a chiffré et envoyé la clé AES à Bob.")

# Bob déchiffre la clé AES avec sa clé privée RSA
decrypted_aes_key = decrypt_rsa(bob_keypair, encrypted_aes_key)
print("Bob a déchiffré la clé AES.")

# Alice envoie un message à Bob
message = b"Hello Bob!"
iv, encrypted_message = encrypt_aes(aes_key, message)
print("Alice a chiffré et envoyé le message à Bob.")

# Bob déchiffre le message
decrypted_message = decrypt_aes(decrypted_aes_key, iv, encrypted_message)
print("Bob a déchiffré le message:", decrypted_message)

# Alice signe le message
signature = sign_data(alice_keypair, message)
print("Alice a signé le message.")

# Bob vérifie la signature d'Alice
verification_result = verify_signature(alice_keypair.public_key(), signature, message)
print("Bob a vérifié la signature d'Alice. Résultat de la vérification:", verification_result)

